In [1]:
import pandas as pd
import numpy as np
import sympy as sym
import math

In [2]:
p = pd.read_csv("./ss.csv")
# p = p.head(20)
# p = p.reset_index(drop=True)

In [3]:
def n_choose_i(n, i):
    """
    :param n: the sympy variable in which the sequence is to be defined
    :param i: the integer to choose from
    :return: the term corresponding to n_choose_i
    """
    expr = 1
    for l in range(i):
        expr *= (n - l)
    return expr / sym.factorial(i)

In [4]:
def construct_polynomial_sequence(seq_df, approximate=False):
    """
    Uses the calculus of sequences along with the Newton-Gregory interpolation formula to determine "what comes next"
    given an arbitrary polynomial sequence
    :param seq_df: a dataframe containing only the sequence in a column "diff_0"
    :param approximate: if there is no valid polynomial that defines the sequence it may still be approximated
    :return: sympy polynomial representation of the sequence
    """

    k = 1
    has_polynomial_representation = False

    while k < len(seq_df) - 1:
        seq_df[f"diff_{k}"] = seq_df[f"diff_{k - 1}"].diff()
        valid = seq_df[f"diff_{k}"].first_valid_index()
        if seq_df[f"diff_{k}"].iloc[valid] == seq_df[f"diff_{k}"].iloc[valid + 1] and seq_df[f"diff_{k}"].iloc[valid] == seq_df[f"diff_{k}"].iloc[valid + 2]:
            has_polynomial_representation = True
            break
        k += 1

    # return seq_df

    if not has_polynomial_representation:
        if not approximate:
            raise Exception("No valid polynomial definition or not enough terms provided for the given sequence")

    leading_terms = []
    for col in seq_df.columns:
        valid_index = seq_df[col].first_valid_index()
        leading_terms.append(seq_df[col].iloc[valid_index])

    n = sym.symbols("n")
    polynomial = 0
    for i, term in enumerate(leading_terms):
        polynomial += term * n_choose_i(n, i)

    return sym.expand(polynomial)

In [7]:
poly = construct_polynomial_sequence(p)
poly

16.6666666666667*n**3 - 50.0*n**2 + 133.333333333333*n

In [9]:
n = sym.symbols("n")

In [39]:
p["index"] = p.index
p["ans"] = p["index"].apply(lambda x: math.ceil(poly.evalf(subs={n: x})))
p = p[["diff_0", "ans"]]
p["descr"] = p["ans"] - p["diff_0"]
p

<ipython-input-39-58be544ff193>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p["descr"] = p["ans"] - p["diff_0"]


,diff_0,ans,descr
0,0.000000e+00,0,0
1,1.000000e+02,100,0
2,2.000000e+02,200,0
3,4.000000e+02,400,0
4,8.000000e+02,800,0
...,...,...,...
495,2.009271e+09,2009271000,0
496,2.021498e+09,2021497600,0
497,2.033774e+09,2033773700,0
498,2.046099e+09,2046099400,0


In [40]:
ans = int(math.ceil(poly.evalf(subs={n: 666666665})))

In [41]:
ans

4938271545679012126374494208

In [36]:
bin(ans)[2:]

'11111111010011010111101011001110100001001101000111101000000000000000000000000000000000000000'

In [42]:
poly

16.6666666666667*n**3 - 50.0*n**2 + 133.333333333333*n

In [ ]:
7638271545679012348271604500
4938271545679012126374494208